In [99]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs
plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
airline_data = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/airline_data.csv", parse_dates = ["date"])
cleaned_reviews = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/cleaned-reviews.csv", parse_dates = ["date"]) # todos los que no opinaron se filtraron ????/
costumer_data = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/customer_airways_data.csv", encoding = "iso-8859-1")
costumer_data_filtrada = pd.read_csv("/content/drive/MyDrive/tp1- orga de datos/Datos clientes/filtered_customer_booking.csv")
costumer_data = costumer_data.drop_duplicates()#eliminamos filas duplicadas

In [101]:
preferencias = costumer_data.iloc[:,8:12]
preferencias = preferencias.rename(columns={"booking_origin": "country"})

In [102]:
def corregir_rates(serie):
  if(serie == "None"):
    return -1
  elif(serie == "\n\t\t\t\t\t\t\t\t\t\t\t\t\t5"):
    return 5
  else:
    return serie

airline_data.rates = airline_data.rates.map(corregir_rates)
airline_data.rates = airline_data.rates.astype(np.int8)

In [ ]:
datos_filtrados = airline_data.iloc[:,[2,4]]
datos_encuesta = pd.merge(datos_filtrados,preferencias,how = "outer")

datos_encuesta_validos =datos_encuesta[datos_encuesta.rates > 0 ]#consideramos que 9 es un puntaje alto
datos_encuesta_completos = datos_encuesta_validos.dropna()
datos_encuesta_completos["country"].value_counts()

conteo_paises = datos_encuesta_completos['country'].value_counts().reset_index()
conteo_paises.columns = ['country', 'conteo_pais']
conteo_paises = conteo_paises.head(5)


In [105]:
promedio_por_paises = datos_encuesta_completos.groupby('country')[['wants_in_flight_meals', 'wants_extra_baggage', 'wants_preferred_seat', 'rates']].mean().reset_index()

promedio_por_paises = promedio_por_paises.sort_values("rates", ascending=False)

paises_mas_encuestas_con_promedios = conteo_paises.merge(promedio_por_paises, on='country', how='inner')
paises_mas_encuestas_con_promedios = paises_mas_encuestas_con_promedios.iloc[:,[0,2,3,4,5]]

In [ ]:
paises_mas_encuestas_con_promedios.set_index('country', inplace=True)
sns.heatmap(paises_mas_encuestas_con_promedios, annot=True, cmap="YlGnBu", fmt="g")

plt.title('Heatmap de preferencias por país')
plt.xlabel('Promedio de preferencia por pais')
plt.ylabel('Países')

# Mostramos el heatmap
plt.show()

# **Analisis de los destinos mas visitados y el tipo de viaje que eligieron**

In [107]:
#esstadistica de rutas preferidas segun fecha -> clasificar fecha por cuatrimestre #suponemos que reservan desde su pais de origen
informacion_de_compra = costumer_data_filtrada[["route", "trip_type", "sales_channel", "booking_complete", "purchase_lead", "booking_origin"]]
informacion_de_compra = informacion_de_compra.rename(columns={"booking_origin": "country"})
fecha_de_viaje = cleaned_reviews[["date", "verified","country"]]
informacion_de_vuelo = pd.merge(informacion_de_compra, fecha_de_viaje, how="outer")

In [109]:
informacion_de_vuelo = informacion_de_vuelo.dropna()
vuelos_validos = informacion_de_vuelo.loc[(informacion_de_vuelo.booking_complete == 1) & (informacion_de_vuelo.verified == True)]

In [110]:
def mes_a_cuatrimestre(fecha):
  if 1 <= fecha.month <= 3:
    return 1
  elif 4 <= fecha.month <= 6:
    return 2
  elif 7 <= fecha.month <= 9:
    return 3
  else:
    return 4

vuelos_validos.date = vuelos_validos.date.map(mes_a_cuatrimestre)
vuelos_validos = vuelos_validos.rename(columns={"date": "quarter"})

In [ ]:
#agrupamos por pais de origen -> vemos que ruta es la mas usada para cada pais y que tipo de venta se uso
paises_y_rutas = vuelos_validos.groupby(["country", "route","trip_type"]).size()
paises_y_rutas = pd.DataFrame(paises_y_rutas)
paises_y_rutas.columns =["cantidad_de_personas"]
paises_y_rutas.reset_index(inplace = True)
paises_y_rutas


In [170]:
vuelos_preferidos =paises_y_rutas.sort_values(["cantidad_de_personas"],ascending=False)

maxima_cantidad_personas = vuelos_preferidos.reset_index().groupby('country')['cantidad_de_personas'].idxmax()

ruta_mas_elegida_pais = vuelos_preferidos.reset_index().loc[maxima_cantidad_personas]

ruta_mas_elegida_pais = ruta_mas_elegida_pais.sort_values("cantidad_de_personas", ascending= False)
ruta_mas_elegida_pais = ruta_mas_elegida_pais.iloc[:,1:]
ruta_mas_elegida_pais

,country,route,trip_type,cantidad_de_personas
0,Malaysia,AKLKUL,RoundTrip,4620
2,United Kingdom,AKLKUL,RoundTrip,2232
5,Australia,SGNSYD,RoundTrip,1140
8,Thailand,DMKKIX,RoundTrip,795
10,Singapore,ICNSIN,RoundTrip,729
14,United States,DPSPVG,RoundTrip,685
70,China,DPSPVG,RoundTrip,304
91,Taiwan,PENTPE,RoundTrip,226
118,Hong Kong,HKGMEL,RoundTrip,152
192,South Korea,BKIICN,RoundTrip,118


In [ ]:
top_10_rutas_elegidas = ruta_mas_elegida_paisvalue_counts

In [ ]:
# Separar la columna "route" en dos columnas: origen y destino
#paises_y_rutas['origen'] = paises_y_rutas['route'].str.slice(0, 3)  # Primeros tres caracteres
#paises_y_rutas['destino'] = paises_y_rutas['route'].str.slice(3)
#paises_y_rutas.drop(columns=['route'], inplace=True)


In [ ]:
#ruta_de_turistas= paises_y_rutas.loc[paises_y_rutas["trip_type"] == "CircleTrip",:]
#ruta_de_turistas
#ponemos los 5 paises mas visitados y que tipo de viaje se uso y que ruta
#destino_preferido = vuelos_preferidos["destino"].value_counts().head()

# Destinos mas visitados -> agrupar por destino
# Paises